In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np

In [3]:
import os
import pickle

# define the directory where the pickle files are stored
folder_path = '../data/batched_data_pickle_files/'

# define the filenames for the pickle files
file_names = ['processed_train_data.pkl', 'processed_train_targets.pkl', 'processed_test_data.pkl', 'true_rul.pkl']

# loop through each file and load its contents as arrays
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    
    # read the pickle file
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
    
    # ensure data is a numpy array
    if isinstance(data, np.ndarray):
        globals()[file_name.replace('.pkl', '')] = data
    else:
        globals()[file_name.replace('.pkl', '')] = np.array(data)

# print the shapes of the loaded data
print("processed train data shape:", processed_train_data.shape)
print("processed train target shape:", processed_train_targets.shape)
print("processed test data shape:", processed_test_data.shape)
print("true rul shape:", true_rul.shape)


processed train data shape: (17731, 30, 14)
processed train target shape: (17731,)
processed test data shape: (100, 30, 14)
true rul shape: (100,)


In [4]:
X = processed_train_data
y = processed_train_targets

# Convert to PyTorch tensors
X_tensor = torch.FloatTensor(X)
y_tensor = torch.FloatTensor(y)

In [6]:
# Split the data
X_train, X_val, y_train, y_val = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

# Create DataLoaders
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)

In [65]:
class SDAE(nn.Module):
    def __init__(self, input_dim, hidden_dims, dropout_rate):
        super(SDAE, self).__init__()
        
        self.input_dim = input_dim
        self.hidden_dims = hidden_dims
        
        # Encoder
        encoder_layers = []
        for i in range(len(hidden_dims)):
            if i == 0:
                encoder_layers.append(nn.Linear(input_dim, hidden_dims[i]))
            else:
                encoder_layers.append(nn.Linear(hidden_dims[i-1], hidden_dims[i]))
            encoder_layers.append(nn.ReLU())
            encoder_layers.append(nn.Dropout(dropout_rate))
        self.encoder = nn.Sequential(*encoder_layers)
        
        # Decoder
        decoder_layers = []
        for i in range(len(hidden_dims)-1, -1, -1):
            if i == 0:
                decoder_layers.append(nn.Linear(hidden_dims[i], input_dim))
            else:
                decoder_layers.append(nn.Linear(hidden_dims[i], hidden_dims[i-1]))
            decoder_layers.append(nn.ReLU())
            decoder_layers.append(nn.Dropout(dropout_rate))
        self.decoder = nn.Sequential(*decoder_layers)
        
        # Predictor
        self.predictor = nn.Linear(hidden_dims[-1], 1)
        
    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        prediction = self.predictor(encoded)
        return decoded, prediction.squeeze()

def add_noise(x, noise_factor=0.5):
    noise = torch.randn_like(x) * noise_factor
    return x + noise

In [77]:
import torch
import torch.optim as optim
import torch.nn as nn
import optuna
import numpy as np

# RMSE loss function
def rmse_loss(predictions, targets):
    return torch.sqrt(nn.MSELoss()(predictions, targets))

# Add noise function (this is a placeholder, define as needed)
def add_noise(x, noise_factor=0.3):
    noise = torch.randn_like(x) * noise_factor
    return x + noise

# Train function to use in Optuna
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    model.to(device)
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            batch_x_noisy = add_noise(batch_x)  # Adding noise for denoising autoencoder
            optimizer.zero_grad()
            decoded, prediction = model(batch_x_noisy)
            loss = criterion(decoded, batch_x.view(batch_x.size(0), -1)) + criterion(prediction, batch_y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch_x, batch_y in val_loader:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                decoded, prediction = model(batch_x)
                loss = criterion(decoded, batch_x.view(batch_x.size(0), -1)) + criterion(prediction, batch_y)
                val_loss += loss.item()
        
        val_loss /= len(val_loader)
    return val_loss  # Return validation loss for Optuna

# Objective function for Optuna to optimize
def objective(trial):
    # Hyperparameter space defined for Optuna
    hidden_dims = [trial.suggest_int('hidden_dim_1', 64, 512, step=64),
                   trial.suggest_int('hidden_dim_2', 32, 256, step=64),
                   trial.suggest_int('hidden_dim_3', 16, 128, step=32)]
    
    # Set dropout rate from 0 to 0.3
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.0, 0.3)
    
    # Log-uniform search for learning rate
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)

    # Create model with the suggested hyperparameters
    model = SDAE(input_dim=30 * 14, hidden_dims=hidden_dims, dropout_rate=dropout_rate)
    criterion = rmse_loss  # Use RMSE loss
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Train the model and get the validation loss
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    val_loss = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20, device=device)

    # Return the validation loss for Optuna to minimize
    return val_loss

# Create Optuna study and optimize
study = optuna.create_study(direction='minimize')  # Minimize the validation loss
study.optimize(objective, n_trials=100)  # Number of trials for hyperparameter search

# Print the best hyperparameters and validation loss
print("Best Hyperparameters:", study.best_params)
print("Best Validation RMSE:", study.best_value)

# Once the best trial is found, save the model with best hyperparameters
best_trial = study.best_trial
best_hidden_dims = [best_trial.params['hidden_dim_1'], best_trial.params['hidden_dim_2'], best_trial.params['hidden_dim_3']]
best_dropout_rate = best_trial.params['dropout_rate']
best_lr = best_trial.params['learning_rate']

# Instantiate the model with best hyperparameters
best_model = SDAE(input_dim=30 * 14, hidden_dims=best_hidden_dims, dropout_rate=best_dropout_rate)
optimizer = optim.Adam(best_model.parameters(), lr=best_lr)
criterion = rmse_loss

# Train the best model on the full training set (optional)
train_model(best_model, train_loader, val_loader, criterion, optimizer, num_epochs=20, device=torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Save the best model to a file
torch.save(best_model.state_dict(), 'best_sdae_model_optuna.pth')
print("Best model saved to 'best_sdae_model_optuna.pth'")


[I 2024-11-14 06:53:18,271] A new study created in memory with name: no-name-d120e809-c307-4af6-aa15-0686601b39c8
/home/ec2-user/.local/lib/python3.9/site-packages/optuna/distributions.py:704: UserWarning: The distribution is specified by [32, 256] and step=64, but the range is not divisible by `step`. It will be replaced by [32, 224].
  warnings.warn(
/home/ec2-user/.local/lib/python3.9/site-packages/optuna/distributions.py:704: UserWarning: The distribution is specified by [16, 128] and step=32, but the range is not divisible by `step`. It will be replaced by [16, 112].
  warnings.warn(
/tmp/ipykernel_155328/1183083817.py:53: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.0, 0.3)
/tmp/ipykernel_155328/1183083817.py:56: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This fe

Best Hyperparameters: {'hidden_dim_1': 64, 'hidden_dim_2': 160, 'hidden_dim_3': 16, 'dropout_rate': 0.00669432238387347, 'learning_rate': 0.0007780335047167862}
Best Validation RMSE: 13.87487622669765
Best model saved to 'best_sdae_model_optuna.pth'


In [80]:
X_test = processed_test_data  # Your processed test data
y_test = true_rul  # Your true RUL values

# Convert to PyTorch tensors
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test)

# Create DataLoader for test data
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Define the best hyperparameters from your Optuna study
# Replace these with the best hyperparameters you found from Optuna
best_hidden_dims = [64, 160, 16]  # Example, replace with your best params from the study
best_dropout_rate = 0.00669432238387347  # Example, replace with your best dropout rate

# Instantiate the best model with saved hyperparameters
best_model = SDAE(input_dim=30 * 14, hidden_dims=best_hidden_dims, dropout_rate=best_dropout_rate)

# Load the best model weights
best_model.load_state_dict(torch.load('best_sdae_model_optuna.pth', map_location='cpu'))  # Use 'cuda' if running on GPU

# Evaluate the model on test data and get predictions
def get_predictions(model, test_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    
    all_predictions = []
    
    with torch.no_grad():
        for batch_x, _ in test_loader:
            batch_x = batch_x.to(device)
            _, predictions = model(batch_x)
            all_predictions.append(predictions.cpu().numpy())
    
    # Concatenate all predictions into a single array
    all_predictions = np.concatenate(all_predictions, axis=0)
    
    return all_predictions

# Get predictions for the test set
predictions = get_predictions(best_model, test_loader)

# Print or inspect the predictions
print(predictions[:10])  # Print the first 10 predictions


[124.17116  118.44553   44.272762  83.81734  102.04889  105.49061
  98.62709  103.0102   124.28079   82.37604 ]


/tmp/ipykernel_155328/2141606109.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load('best_sdae_model_optuna.pth', map_location='cpu')

In [81]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

def evaluate_rul_metrics(true, predicted):
    
    true = np.array(true)
    predicted = np.array(predicted)
    
    mae = float(mean_absolute_error(true, predicted))
    mse = float(mean_squared_error(true, predicted))
    rmse = float(np.sqrt(mse))
    mape = float(np.mean(np.abs((true - predicted) / true)) * 100)
    
    return {
        "MAE": mae,
        "MSE": mse,
        "RMSE": rmse,
        "MAPE": mape
    }
    
metrics = evaluate_rul_metrics(true_rul, predictions)
metrics

{'MAE': 11.506507139205933,
 'MSE': 223.9141398666582,
 'RMSE': 14.963760886443561,
 'MAPE': 18.865841994164203}

In [20]:
import torch
import torch.optim as optim
import torch.nn as nn
import optuna
import numpy as np

# RMSE loss function
def rmse_loss(predictions, targets):
    return torch.sqrt(nn.MSELoss()(predictions, targets))

# Add noise function (this is a placeholder, define as needed)
def add_noise(x, noise_factor=0.3):
    noise = torch.randn_like(x) * noise_factor
    return x + noise

# Train function to use in Optuna
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    model.to(device)
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            batch_x_noisy = add_noise(batch_x)  # Adding noise for denoising autoencoder
            optimizer.zero_grad()
            decoded, prediction = model(batch_x_noisy)
            loss = criterion(decoded, batch_x.view(batch_x.size(0), -1)) + criterion(prediction, batch_y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch_x, batch_y in val_loader:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                decoded, prediction = model(batch_x)
                loss = criterion(decoded, batch_x.view(batch_x.size(0), -1)) + criterion(prediction, batch_y)
                val_loss += loss.item()
        
        val_loss /= len(val_loader)
    return val_loss  # Return validation loss for Optuna

# Objective function for Optuna to optimize
def objective(trial):
    # Hyperparameter space defined for Optuna
    hidden_dims = [trial.suggest_int('hidden_dim_1', 64, 256, step=32),
                   trial.suggest_int('hidden_dim_2', 32, 128, step=32),
                   trial.suggest_int('hidden_dim_3', 16, 128, step=32)]
    
    # Set dropout rate from 0 to 0.3
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.0, 0.3)
    
    # Log-uniform search for learning rate
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)

    # Create model with the suggested hyperparameters
    model = SDAE(input_dim=30 * 14, hidden_dims=hidden_dims, dropout_rate=dropout_rate)
    criterion = rmse_loss  # Use RMSE loss
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Train the model and get the validation loss
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    val_loss = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20, device=device)

    # Return the validation loss for Optuna to minimize
    return val_loss

# Create Optuna study and optimize
study = optuna.create_study(direction='minimize')  # Minimize the validation loss
study.optimize(objective, n_trials=100)  # Number of trials for hyperparameter search

# Print the best hyperparameters and validation loss
print("Best Hyperparameters:", study.best_params)
print("Best Validation RMSE:", study.best_value)

# Once the best trial is found, save the model with best hyperparameters
best_trial = study.best_trial
best_hidden_dims = [best_trial.params['hidden_dim_1'], best_trial.params['hidden_dim_2'], best_trial.params['hidden_dim_3']]
best_dropout_rate = best_trial.params['dropout_rate']
best_lr = best_trial.params['learning_rate']

# Instantiate the model with best hyperparameters
best_model = SDAE(input_dim=30 * 14, hidden_dims=best_hidden_dims, dropout_rate=best_dropout_rate)
optimizer = optim.Adam(best_model.parameters(), lr=best_lr)
criterion = rmse_loss

# Train the best model on the full training set (optional)
train_model(best_model, train_loader, val_loader, criterion, optimizer, num_epochs=20, device=torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Save the best model to a file
torch.save(best_model.state_dict(), 'best_sdae_model_optuna_second.pth')
print("Best model saved to 'best_sdae_model_optuna_second.pth'")

[I 2024-11-14 04:04:56,892] A new study created in memory with name: no-name-3b6a0f5b-ac39-493f-a284-b8b3457e4fe7
/tmp/ipykernel_155328/2588598532.py:53: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.0, 0.3)
/tmp/ipykernel_155328/2588598532.py:56: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
[I 2024-11-14 04:05:13,336] Trial 0 finished with value: 16.609278917312622 and parameters: {'hidden_dim_1': 160, 'hidden_dim_2': 64, 'hidden_dim_3': 80, 'dropout_rate': 0.15922720170060026, 'learning_rate': 0.00022075549335688925}. Best is trial 0 with value: 16.6092789173126

Best Hyperparameters: {'hidden_dim_1': 64, 'hidden_dim_2': 64, 'hidden_dim_3': 80, 'dropout_rate': 0.05155521210111333, 'learning_rate': 0.0009057370211032963}
Best Validation RMSE: 14.186381884983607
Best model saved to 'best_sdae_model_optuna_second.pth'


In [89]:
X_test = processed_test_data  # Your processed test data
y_test = true_rul  # Your true RUL values

# Convert to PyTorch tensors
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test)

# Create DataLoader for test data
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Define the best hyperparameters from your Optuna study
# Replace these with the best hyperparameters you found from Optuna
best_hidden_dims = [64, 64, 80]  # Example, replace with your best params from the study
best_dropout_rate = 0.05155521210111333  # Example, replace with your best dropout rate

# Instantiate the best model with saved hyperparameters
best_model = SDAE(input_dim=30 * 14, hidden_dims=best_hidden_dims, dropout_rate=best_dropout_rate)

# Load the best model weights
best_model.load_state_dict(torch.load('best_sdae_model_optuna_second.pth', map_location='cpu'))  # Use 'cuda' if running on GPU

# Evaluate the model on test data and get predictions
def get_predictions(model, test_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    
    all_predictions = []
    
    with torch.no_grad():
        for batch_x, _ in test_loader:
            batch_x = batch_x.to(device)
            _, predictions = model(batch_x)
            all_predictions.append(predictions.cpu().numpy())
    
    # Concatenate all predictions into a single array
    all_predictions = np.concatenate(all_predictions, axis=0)
    
    return all_predictions

# Get predictions for the test set
predictions = get_predictions(best_model, test_loader)

# Print or inspect the predictions
print(predictions[:10])  # Print the first 10 predictions


[108.270035 123.95801   46.380363  89.536026 101.55478  105.04238
  88.46287   98.11394  117.86893   71.83512 ]


/tmp/ipykernel_155328/3303680736.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load('best_sdae_model_optuna_second.pth', map_location

In [90]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

def evaluate_rul_metrics(true, predicted):
    
    true = np.array(true)
    predicted = np.array(predicted)
    
    mae = float(mean_absolute_error(true, predicted))
    mse = float(mean_squared_error(true, predicted))
    rmse = float(np.sqrt(mse))
    mape = float(np.mean(np.abs((true - predicted) / true)) * 100)
    
    return {
        "MAE": mae,
        "MSE": mse,
        "RMSE": rmse,
        "MAPE": mape
    }
    
metrics = evaluate_rul_metrics(true_rul, predictions)
metrics

{'MAE': 10.44703010559082,
 'MSE': 205.7282444988759,
 'RMSE': 14.343229918636734,
 'MAPE': 16.14549992434262}

In [91]:
import torch
import torch.optim as optim
import torch.nn as nn
import optuna
import numpy as np

# RMSE loss function
def rmse_loss(predictions, targets):
    return torch.sqrt(nn.MSELoss()(predictions, targets))

# Add noise function (this is a placeholder, define as needed)
def add_noise(x, noise_factor=0.3):
    noise = torch.randn_like(x) * noise_factor
    return x + noise

# SDAE Model (without dropout)
class SDAE(nn.Module):
    def __init__(self, input_dim, hidden_dims):
        super(SDAE, self).__init__()

        # Encoder layers
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dims[0]),
            nn.ReLU(),
            nn.Linear(hidden_dims[0], hidden_dims[1]),
            nn.ReLU(),
            nn.Linear(hidden_dims[1], hidden_dims[2]),
            nn.ReLU(),
        )
        
        # Decoder layers
        self.decoder = nn.Sequential(
            nn.Linear(hidden_dims[2], hidden_dims[1]),
            nn.ReLU(),
            nn.Linear(hidden_dims[1], hidden_dims[0]),
            nn.ReLU(),
            nn.Linear(hidden_dims[0], input_dim),
            nn.Sigmoid(),  # Sigmoid to keep values between [0, 1] for reconstruction
        )
        
        # Predictor layer for RUL prediction
        self.predictor = nn.Linear(hidden_dims[2], 1)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        prediction = self.predictor(encoded)
        return decoded, prediction.squeeze()

# Train function to use in Optuna
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    model.to(device)
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            batch_x_noisy = add_noise(batch_x)  # Adding noise for denoising autoencoder
            optimizer.zero_grad()
            decoded, prediction = model(batch_x_noisy)
            loss = criterion(decoded, batch_x.view(batch_x.size(0), -1)) + criterion(prediction, batch_y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch_x, batch_y in val_loader:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                decoded, prediction = model(batch_x)
                loss = criterion(decoded, batch_x.view(batch_x.size(0), -1)) + criterion(prediction, batch_y)
                val_loss += loss.item()
        
        val_loss /= len(val_loader)
    return val_loss  # Return validation loss for Optuna

# Objective function for Optuna to optimize
def objective(trial):
    # Hyperparameter space defined for Optuna
    hidden_dims = [trial.suggest_int('hidden_dim_1', 64, 512, step=64),
                   trial.suggest_int('hidden_dim_2', 32, 256, step=64),
                   trial.suggest_int('hidden_dim_3', 16, 128, step=32)]
    
    # Log-uniform search for learning rate
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)

    # Create model with the suggested hyperparameters
    model = SDAE(input_dim=30 * 14, hidden_dims=hidden_dims)
    criterion = rmse_loss  # Use RMSE loss
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Train the model and get the validation loss
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    val_loss = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20, device=device)

    # Return the validation loss for Optuna to minimize
    return val_loss

# Create Optuna study and optimize
study = optuna.create_study(direction='minimize')  # Minimize the validation loss
study.optimize(objective, n_trials=100)  # Number of trials for hyperparameter search

# Print the best hyperparameters and validation loss
print("Best Hyperparameters:", study.best_params)
print("Best Validation RMSE:", study.best_value)

# Once the best trial is found, save the model with best hyperparameters
best_trial = study.best_trial
best_hidden_dims = [best_trial.params['hidden_dim_1'], best_trial.params['hidden_dim_2'], best_trial.params['hidden_dim_3']]
best_lr = best_trial.params['learning_rate']

# Instantiate the model with best hyperparameters
best_model = SDAE(input_dim=30 * 14, hidden_dims=best_hidden_dims)
optimizer = optim.Adam(best_model.parameters(), lr=best_lr)
criterion = rmse_loss

# Train the best model on the full training set (optional)
train_model(best_model, train_loader, val_loader, criterion, optimizer, num_epochs=20, device=torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Save the best model to a file
torch.save(best_model.state_dict(), 'best_sdae_model_optuna_no_dropout.pth')
print("Best model saved to 'best_sdae_model_optuna_no_dropout.pth'")


[I 2024-11-14 07:56:48,837] A new study created in memory with name: no-name-6bca8cc1-ba8d-4f3b-8cc0-480149aa645d
/home/ec2-user/.local/lib/python3.9/site-packages/optuna/distributions.py:704: UserWarning: The distribution is specified by [32, 256] and step=64, but the range is not divisible by `step`. It will be replaced by [32, 224].
  warnings.warn(
/home/ec2-user/.local/lib/python3.9/site-packages/optuna/distributions.py:704: UserWarning: The distribution is specified by [16, 128] and step=32, but the range is not divisible by `step`. It will be replaced by [16, 112].
  warnings.warn(
/tmp/ipykernel_155328/2979645140.py:88: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
[I 2024-11-14 07:57:18,874] Trial 0 finished with value: 19.6072142464774 and parameter

Best Hyperparameters: {'hidden_dim_1': 64, 'hidden_dim_2': 96, 'hidden_dim_3': 16, 'learning_rate': 0.0007835726980214638}
Best Validation RMSE: 13.731650062969752
Best model saved to 'best_sdae_model_optuna_no_dropout.pth'


In [109]:
# Instantiate the model with best hyperparameters
best_model = SDAE(input_dim=30 * 14, hidden_dims=best_hidden_dims)
optimizer = optim.Adam(best_model.parameters(), lr=best_lr)
criterion = rmse_loss

# Train the best model on the full training set (optional)
train_model(best_model, train_loader, val_loader, criterion, optimizer, num_epochs=20, device=torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Save the best model to a file
torch.save(best_model.state_dict(), 'best_sdae_model_optuna_no_dropout.pth')
print("Best model saved to 'best_sdae_model_optuna_no_dropout.pth'")

Best model saved to 'best_sdae_model_optuna_no_dropout.pth'


In [110]:
X_test = processed_test_data  # Your processed test data
y_test = true_rul  # Your true RUL values

# Convert to PyTorch tensors
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test)

# Create DataLoader for test data
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Define the best hyperparameters from your Optuna study
# Replace these with the best hyperparameters you found from Optuna
best_hidden_dims = [64, 96, 16]  # Example, replace with your best params from the study

# Instantiate the best model with saved hyperparameters
best_model = SDAE(input_dim=30 * 14, hidden_dims=best_hidden_dims)

# Load the best model weights
best_model.load_state_dict(torch.load('best_sdae_model_optuna_no_dropout.pth', map_location='cpu'))  # Use 'cuda' if running on GPU

# Evaluate the model on test data and get predictions
def get_predictions(model, test_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    
    all_predictions = []
    
    with torch.no_grad():
        for batch_x, _ in test_loader:
            batch_x = batch_x.to(device)
            _, predictions = model(batch_x)
            all_predictions.append(predictions.cpu().numpy())
    
    # Concatenate all predictions into a single array
    all_predictions = np.concatenate(all_predictions, axis=0)
    
    return all_predictions

# Get predictions for the test set
predictions = get_predictions(best_model, test_loader)

# Print or inspect the predictions
print(predictions[:10])  # Print the first 10 predictions


[114.84518  123.111885  45.846992  85.572945  93.635666 108.51436
 105.75062   88.21238  112.03609   89.69039 ]


/tmp/ipykernel_155328/3708898075.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load('best_sdae_model_optuna_no_dropout.pth', map_loca

In [111]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

def evaluate_rul_metrics(true, predicted):
    
    true = np.array(true)
    predicted = np.array(predicted)
    
    mae = float(mean_absolute_error(true, predicted))
    mse = float(mean_squared_error(true, predicted))
    rmse = float(np.sqrt(mse))
    mape = float(np.mean(np.abs((true - predicted) / true)) * 100)
    
    return {
        "MAE": mae,
        "MSE": mse,
        "RMSE": rmse,
        "MAPE": mape
    }
    
metrics = evaluate_rul_metrics(true_rul, predictions)
metrics

{'MAE': 10.99563497543335,
 'MSE': 213.497219073085,
 'RMSE': 14.611544034532592,
 'MAPE': 17.06540950254447}